In [8]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text 
n = 10000
path = f"desc_data_{n}.csv"
data = pd.read_csv(path)
seff_cols = [i for i in data.columns if "sideEffect" in i]


In [9]:
print([i for i in data.columns if "sideEffect" not in i])

['id', 'name', 'substitute0', 'substitute1', 'substitute2', 'substitute3', 'substitute4', 'use0', 'use1', 'use2', 'use3', 'use4', 'Chemical Class', 'Habit Forming', 'Therapeutic Class', 'Action Class', 'descriptions']


In [10]:

def generate_data(data,scols):
    doc_dict = {}
    for col in scols:
        for n,row in enumerate(data[col]):
            if(row not in doc_dict.keys()):
                doc_dict[row] = []
            if(row):
                doc_dict[row].append(data["descriptions"][n])
    return doc_dict
doc_dict = generate_data(data,seff_cols)


In [11]:

import json
def get_top_tfidf_terms(documents):
    # Concatenate the texts in each document
    texts = [' '.join(text_list) for text_list in documents.values()]

    additional_stop_words = ["combination","contains","medicines","medicine","effects","belongs"] 

    stop_words = text.ENGLISH_STOP_WORDS.union(additional_stop_words)

    tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)

    tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

    feature_names = tfidf_vectorizer.get_feature_names_out()

    top_terms_per_doc = {}
    for doc_id, doc_text in documents.items():
        # Find the TF-IDF vector for the document
        doc_index = list(documents.keys()).index(doc_id)
        tfidf_vector = tfidf_matrix[doc_index]

        top_indices = tfidf_vector.indices[np.argsort(tfidf_vector.data)[-10:]][::-1]

        # Extract terms with 7 letters or more and their corresponding TF-IDF scores
        top_terms = [(feature_names[idx], tfidf_vector[0, idx]) for idx in top_indices if len(feature_names[idx]) >= 7]

        top_terms_per_doc[doc_id] = top_terms

    return top_terms_per_doc

top_terms_per_doc = get_top_tfidf_terms(doc_dict)

# Print top TF-IDF terms for each document



In [12]:
#word based ordering
idf_dict = {}
side_dict = {}

for doc_id, top_terms in top_terms_per_doc.items():
    if str(doc_id) != 'nan':
        for term, tfidf_score in top_terms:
            if(term in side_dict.keys()):
                if format(tfidf_score, '.2f')>idf_dict[term]:
                    idf_dict[term] = format(tfidf_score, '.2f')
                    side_dict[term] = doc_id
            else:
                idf_dict[term] =  format(tfidf_score, '.2f')
                side_dict[term] = doc_id



In [13]:
print(side_dict["antihistamines"])

Fear


In [7]:
with open('sideeffect_dict.json', 'w') as f:
    json.dump(side_dict, f)
with open('idf_score.json', 'w') as f:
    json.dump(idf_dict, f)


In [8]:
for i in list(side_dict.keys()):
    if(i not in idf_dict.keys()):
        print(i)

In [9]:
idf_dict["chemical"]

'0.24'

In [11]:
top_terms_per_doc

{'Vomiting': [('amoxicillin', 0.17156569130815527),
  ('bacteria', 0.15143023838479527),
  ('infections', 0.14241908662309952),
  ('guaifenesin', 0.12509629418437163)],
 'Nausea': [('amoxicillin', 0.12318291094741361),
  ('amlodipine', 0.12280128686543246)],
 'Headache': [('amlodipine', 0.19448619641028567),
  ('ambroxol', 0.13271499773205825),
  ('guaifenesin', 0.12668741341487041)],
 'Sleepiness': [('amlodipine', 0.3241537012115173),
  ('blocker', 0.1813909174688548),
  ('atenolol', 0.1630131724095436),
  ('pressure', 0.15969341739669377),
  ('vessels', 0.1320206230197661),
  ('histamine', 0.12468310040045236),
  ('channel', 0.12132535597322608)],
 'Sedation': [('hydroxyzine', 0.34005207854378433),
  ('anxiety', 0.2425588847648472),
  ('tension', 0.21283847638144915),
  ('antihistamines', 0.18708129101799495),
  ('itching', 0.18518481070764048),
  ('reactions', 0.183138628987611)],
 'Application site reactions (burning, irritation, itching and redness)': [('acyclovir',
   0.164803027

In [10]:
top_terms_per_doc['Vomiting']

[('amoxicillin', 0.17156569130815527),
 ('bacteria', 0.15143023838479527),
 ('infections', 0.14241908662309952),
 ('guaifenesin', 0.12509629418437163)]

In [16]:
import pandas as pd
import numpy as np
csv_data = pd.read_csv("desc_data_10000.csv")
csv_data = csv_data.replace([np.nan], -1)
selected_docs = ["acetylcysteine","acebrophylline","antihistamines","histamine","hydroxyzine","atenolol","guaifenesin","amlodipine","amoxicillin"]
csv_data = csv_data.loc[csv_data['descriptions'].str.contains('|'.join(selected_docs))]
csv_data.reset_index(inplace=True)
csv_data.to_csv("filtered_data.csv")

In [20]:
csv_data["name"]

0          augmentin 625 duo tablet
1              allegra 120mg tablet
2                atarax 25mg tablet
3                 aciloc 150 tablet
4     augmentin duo oral suspension
                  ...              
64                almox 250 capsule
65                      alkof syrup
66                      alkof syrup
67                      alkof syrup
68                      alkof syrup
Name: name, Length: 69, dtype: object